<a href="https://colab.research.google.com/github/AlibekAdilzhan/ml_models/blob/main/nn_v1_for_iris_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
class NeuralNetwork:
    def __init__(self, layers, epoch_number, early_stopping = False, stopping_threshold = 0.1):
        self.layers = layers
        self.weights = []
        self.biases = []
        self.layers_number = len(layers)
        self.errors = []
        self.epoch = epoch_number
        self.early_stopping = False
        self.threshold = stopping_threshold


    def g_fn(self, x):
        z = 1 / (1 + np.exp(-x))

        return z

    def initialize_WB(self):
        for i in range(len(self.layers) - 1):
            W = np.random.rand(self.layers[i + 1], self.layers[i])
            B = np.random.rand(self.layers[i + 1], 1)
            self.biases.append(B)
            self.weights.append(W)

    def mse(self, Y, Y_predicted):
        z = np.sum((Y - Y_predicted)**2)
        return z

    def forward_propogation(self, x):
        A = [0]
        Z = [x.T]

        for i in range(self.layers_number - 1):
            Zi = Z[i]
            # print(Zi)
            Wi = self.weights[i]
            Bi = self.biases[i]
            A_next = Wi.dot(Zi) + Bi
            Z_next = self.g_fn(A_next)

            A.append(A_next)
            Z.append(Z_next)
        
        return A, Z

    def backward_propogation(self, x, y, A, Z):
        y = y.reshape(len(y), 1)
        dLdA = []
        dLdW = []

        A_last = A[-1]
        Z_last = Z[-1]
        Z_pre_last = Z[-2]
        # print(A_last, 'it is A')
        # print(y, 'and that is y')

        dLda_last = -(y - self.g_fn(A_last)) * self.g_fn(A_last) * (1 - self.g_fn(A_last))
        dLdW_last = dLda_last.dot(Z_pre_last.T)

        dLdA.insert(0, dLda_last)
        dLdW.insert(0, dLdW_last)

        for i in list(reversed(range(1, self.layers_number - 1))):
            dLda_next_layer = dLdA[0]
            Ai = A[i]
            Z_previous_layer = Z[i - 1]
            Wi = self.weights[i]
            dLdai = self.g_fn(Ai) * (1 - self.g_fn(Ai)) * Wi.T.dot(dLda_next_layer)
            dLdWi = dLdai.dot(Z_previous_layer.T)

            dLdA.insert(0, dLdai)
            dLdW.insert(0, dLdWi)

        return dLdA, dLdW

    def fit(self, X, Y):
        N = len(X)
        alpha = 0.6
        Y_predicted = np.zeros(Y.shape)

        for _ in range(self.epoch):
            for i in range(N):
                x = X[i]
                y = Y[i]
                x = x.reshape((1, self.layers[0]))
                A, Z = self.forward_propogation(x)
                dLdA, dLdW = self.backward_propogation(x, y, A, Z)

                for j in range(len(self.weights)):
                    self.weights[j] = self.weights[j] - alpha * dLdW[j]

                for j in range(len(self.biases)):
                    self.biases[j] = self.biases[j] - alpha * dLdA[j]

                _, Z1 = self.forward_propogation(x)
                Y_predicted[i] = Z1[-1].T[0]
                # print(Y_predicted[i], Y[i])
                current_error = self.mse(Y_predicted[i], Y[i])
                print(current_error)
                # print(Z1[-1].T[0])
            
            error = self.mse(Y, Y_predicted)
            self.errors.append(error)
            
            if self.early_stopping == True:
                if error < self.threshold:
                    break




In [ ]:
from sklearn.datasets import load_iris

data =  load_iris()

X = data['data']
Y = data['target']

In [ ]:
print(Y.shape)

In [ ]:
l = []

for i in range(len(Y)):
    a = np.zeros(3)
    a[Y[i]] = 1
    l.append(a)

Y_one_hot = np.array(l)
print(Y_one_hot[0])

In [ ]:
nn_iris = NeuralNetwork([4, 5, 3], 10000, True)
nn_iris.initialize_WB()

In [ ]:
indices = np.random.permutation(len(X))

X_shuffeled, Y_shuffeled = X[indices], Y_one_hot[indices]
X_train = X_shuffeled[: 120]
Y_train = Y_shuffeled[: 120]
X_test = X_shuffeled[120 : ]
Y_test = Y_shuffeled[120 : ]

In [ ]:
nn_iris.fit(X_train, Y_train)

In [ ]:
nn_iris.errors[-2]

In [ ]:
print(min(nn_iris.errors))

In [ ]:
print(nn_iris.errors)

In [ ]:
error_number = 0

for i in range(len(X_test)):
    _, Z1 = nn_iris.forward_propogation(X_test[i])
    y_predicted = Z1[-1].T[0]
    max_argument = np.argmax(y_predicted)
    if int(Y_test[0][max_argument]) != 1:
        error_number += 1

In [ ]:
print(y_predicted)
print(Y_test[-1])

In [ ]:
print(error_number / len(X_test))

In [ ]:
fig = plt.figure(figsize = (10, 6))

plt.plot(nn_iris.errors[4000 : ])